In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import time
import json
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
from collections import OrderedDict
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms

In [0]:
# we will download the required data files
!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
#!rm -r flower_data || true
!unzip -qq flower_data.zip

In [0]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
#Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

#Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)

#Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=64)

In [0]:
#label mapping
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [47]:
#Build and train your network
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [49]:
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 102),
                                 nn.LogSoftmax(dim=1))

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [64]:
n_epochs = 25

valid_loss_min = np.Inf

for epoch in range(1, n_epochs+1):

    train_loss = 0.0
    valid_loss = 0.0
    
    #training the model
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        # forward pass
        output = model(data)
        loss = criterion(output, target)
        # backward pass
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()*data.size(0)
          
    #validating the model
    model.eval()
    for batch_idx, (data, target) in enumerate(validloader):
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass
        output = model(data)
        loss = criterion(output, target)
        
        valid_loss += loss.item()*data.size(0)
    
    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = valid_loss/len(validloader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))

    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_augmented.pt')
        valid_loss_min = valid_loss
 
    else:
      loss_test = 0
      accuracy = 0
      with torch.no_grad():

          model.eval()

          for images, labels in train_loader:
              log_ps = model(images)
              loss_test += criterion(log_ps, labels)
              ps = torch.exp(log_ps)
              top_p, top_class = ps.topk(1, dim=1)
              equals = top_class == labels.view(*top_class.shape)
              accuracy = torch.mean(equals.type(torch.FloatTensor))

      model.train()  

      print(f'Accuracy: {accuracy.item()*100}%')

Epoch: 1 	Training Loss: 3.070434 	Validation Loss: 2.692481
Validation loss decreased (inf --> 2.692481).  Saving model ...
Epoch: 2 	Training Loss: 2.842314 	Validation Loss: 2.435829
Validation loss decreased (2.692481 --> 2.435829).  Saving model ...
Epoch: 3 	Training Loss: 2.641645 	Validation Loss: 2.221375
Validation loss decreased (2.435829 --> 2.221375).  Saving model ...
Epoch: 4 	Training Loss: 2.456560 	Validation Loss: 2.009407
Validation loss decreased (2.221375 --> 2.009407).  Saving model ...
Epoch: 5 	Training Loss: 2.287395 	Validation Loss: 1.854447
Validation loss decreased (2.009407 --> 1.854447).  Saving model ...
Epoch: 6 	Training Loss: 2.162484 	Validation Loss: 1.676242
Validation loss decreased (1.854447 --> 1.676242).  Saving model ...
Epoch: 7 	Training Loss: 2.002096 	Validation Loss: 1.537566
Validation loss decreased (1.676242 --> 1.537566).  Saving model ...
Epoch: 8 	Training Loss: 1.888170 	Validation Loss: 1.420137
Validation loss decreased (1.53756

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# Save the checkpoint
model_save_name = 'classifier.pt'
model_save_path = F"/content/gdrive/My Drive/Colab Notebooks/Lab_Challenge/{model_save_name}"
torch.save(model.state_dict(), model_save_path)

In [0]:
# Write a function that loads a checkpoint and rebuilds the model
model_save_name = 'classifier.pt'
model_save_path = F"/content/gdrive/My Drive/Colab Notebooks/Lab_Challenge/{model_save_name}"
model.load_state_dict(torch.load(model_save_path))